In [ ]:
# default_exp data

# Data

> API details.

In [ ]:
#export 
from ncempy.io.dm import fileDM
from smpr3d.util import wavelength
from torch.utils.data import Dataset
from skimage.filters import gaussian
from smpr3d.util import crop_symmetric_around_center, sparse_to_dense_datacube_crop, \
    sparse_to_dense_datacube_crop_gain_mask, center_of_mass_kernel, sum_frames, rotate, virtual_annular_image_kernel, \
    fftshift_kernel, fftshift_pad_kernel, dense_to_sparse_kernel, fourier_coordinates_2D, \
    array_split_divpoints_ntotal, beamlet_samples, natural_neighbor_weights

from __future__ import annotations
import sigpy as sp
from numba import cuda
from fastcore.utils import basic_repr
import h5py 
from typing import TYPE_CHECKING, List, NamedTuple, Optional, Union, Callable
import torch as th
import numpy as np 
from numpy.fft import fftshift
from dataclasses import dataclass

In [ ]:
#export 
@dataclass
class DetectorMeta:
    bright_field_center : np.array
    bright_field_radius : float    
    
    def __init__(self, bright_field_center, bright_field_radius):
        self.bright_field_radius = bright_field_radius
        self.bright_field_center = bright_field_center
        
    def to_h5(self, file_path, key = 'detector_meta'):
            with h5py.File(file_path, 'a') as f:
                g = f.create_group(key)
                g.create_dataset('bright_field_radius', data=self.bright_field_radius)
                g.create_dataset('bright_field_center', data=self.bright_field_center)
            
    @staticmethod        
    def from_h5(file_path, key = 'detector_meta'):
        with h5py.File(file_path, 'r') as f:
            g = f[key]
            bright_field_center = g['bright_field_center'][...]
            bright_field_radius = g['bright_field_radius'][()]
        res = DetectorMeta(bright_field_center,bright_field_radius)
        return res 

@dataclass
class SMeta:
    f : np.array
    M : np.array
    N : np.array
    dx : np.array
    
    q : th.tensor
    qf : th.tensor 
    q2 : th.tensor 
    qf2 : th.tensor 
    q_coords : th.tensor
    r_indices : th.tensor
    all_beams : th.tensor
    parent_beams : th.tensor
    beam_numbers : th.tensor
    all_beams_q : th.tensor
    all_beams_coords : th.tensor
    parent_beams_q : th.tensor
    parent_beams_coords : th.tensor
    # natural_neighbor_weights : th.tensor
    # beamlets : th.tensor
    q_dft : th.tensor
    
    numerical_aperture_radius_pixels : int
    B : int 
    Bp : int
    
    device : th.device
    
    
    def __init__(self, 
                 all_beams : np.array,                 
                 parent_beams : np.array,
                 numerical_aperture_radius_pixels,
                 dx : np.array, 
                 N : tuple, 
                 M : tuple, 
                 device : th.device):
        self.dx = dx
        self.device = device
        self.all_beams = th.as_tensor(all_beams).to(device).bool()
        self.parent_beams = th.as_tensor(parent_beams).to(device).bool()
        self.numerical_aperture_radius_pixels = numerical_aperture_radius_pixels
        self.B = int(all_beams.sum())
        self.Bp = int(parent_beams.sum())
        
        MY, MX = M
        NY, NX = N
        self.M = np.array(M)
        self.N = np.array(N)

        self.f = np.array([NY, NX]) // np.array([MY, MX])
        self.q = th.as_tensor(fourier_coordinates_2D([NY, NX], dx, centered=False), device=device)
        self.qf = th.as_tensor(fourier_coordinates_2D([MY, MX], dx, centered=False), device=device)
        self.q2 = th.norm(self.q, dim=0) ** 2
        self.qf2 = th.norm(self.qf, dim=0) ** 2
        mgrid = np.array(np.mgrid[-MY // 2:MY // 2, -MX // 2:MX // 2])
        self.q_coords = th.from_numpy(fftshift(mgrid, (1, 2))).to(device)
        self.r_indices = th.from_numpy(np.mgrid[:MY, :MX]).to(device)
        
        self.beam_numbers = th.ones_like(self.all_beams, dtype=th.long, device=device) * -1
        self.beam_numbers[self.all_beams] = th.arange(0, self.B, device=device)
        
        all_beams_expanded = self.all_beams[None,...].expand_as(self.q_coords)
        self.all_beams_q = self.qf[all_beams_expanded].reshape(2,self.B).T
        self.all_beams_coords = self.q_coords[all_beams_expanded].reshape(2,self.B).T
        
        parent_beams_expanded = self.parent_beams[None,...].expand_as(self.q_coords)
        self.parent_beams_q = self.qf[parent_beams_expanded].reshape(2,self.Bp).T
        self.parent_beams_coords = self.q_coords[parent_beams_expanded].reshape(2,self.Bp).T
        
        self.q_dft = th.from_numpy(fourier_coordinates_2D([MY, MX], [1, 1], centered=False)).to(device)
        
        #S_shape          array (3,)        
        #q              (NY, NX) x
        #qf             (MY, MX) x
        #q2             (NY, NX) x
        #q2f            (MY, MX) x
        #f              (2,)
        #q_coords       (MY, MX)
        #r_indices      (NY, NX)
        #all_beams     (MY, MX)
        #beam_numbers   (MY, MX)
        #q_b            (B, 2)
        #q_b_coords     (B, 2)
        #q_dft          (MY, MX)
        
    def to_h5(self, file_path, key = 's_matrix_meta'):
        with h5py.File(file_path, 'a') as f:
            g = f.create_group(key)
            g.create_dataset('M', data=self.M)
            g.create_dataset('N', data=self.N)
            g.create_dataset('all_beams', data=self.all_beams.cpu().numpy())
            g.create_dataset('parent_beams', data=self.parent_beams.cpu().numpy())
            g.create_dataset('numerical_aperture_radius_pixels', data=self.numerical_aperture_radius_pixels)
            g.create_dataset('dx', data=self.dx)
            g.create_dataset('device', data=str(self.device))
            
    @staticmethod        
    def from_h5(file_path, key = 's_matrix_meta'):
        with h5py.File(file_path, 'r') as f:
            g = f[key]
            all_beams = g['all_beams'][...]
            parent_beams = g['parent_beams'][...]
            numerical_aperture_radius_pixels = g['numerical_aperture_radius_pixels'][()]
            dx = g['dx'][...]
            N = g['N'][...]
            M = g['M'][...]
            device = th.device(str(g['device'][...]))
        res = SMeta(all_beams, parent_beams, numerical_aperture_radius_pixels, dx, N, M, device)
        return res 
    
    def make_beamlet_meta(self, n_radial_samples, n_angular_samples = 6) -> SMeta:
        parent_beams_coords = beamlet_samples(self.all_beams.cpu().numpy(), 
                            self.numerical_aperture_radius_pixels,
                            n_angular_samples, 
                            n_radial_samples) 
        
        parent_beams = th.zeros(tuple(self.M), dtype=th.bool)
        for si in parent_beams_coords:
            parent_beams[si[0],si[1]] = 1
            
        Bp = parent_beams_coords.shape[0]
        mgrid = np.array(np.mgrid[-self.M[0] // 2:self.M[0] // 2, -self.M[1] // 2:self.M[1] // 2])
        q_coords = th.from_numpy(fftshift(mgrid, (1, 2)))
        parent_beams_expanded = parent_beams[None,...].expand_as(q_coords)
        parent_beams_coords = q_coords[parent_beams_expanded].reshape(2,Bp).T
            
        if n_radial_samples > 1:
            nnw = natural_neighbor_weights(parent_beams_coords, 
                                          self.all_beams_coords.cpu().numpy(), 
                                          minimum_weight_cutoff=1e-2)
        else:
            nnw = np.ones((self.B, 1))
            
        beamlets = []
        for j in range(nnw.shape[1]):
            wsample = nnw[:, j]
            ww = np.zeros(self.all_beams.shape, dtype=np.float32)
            ww[self.all_beams_coords[:, 1].cpu().numpy(), 
               self.all_beams_coords[:, 0].cpu().numpy()] = wsample
            beamlets.append(ww)
        beamlets = np.array(beamlets)
            
        new_meta = SMeta(self.all_beams, 
                         parent_beams, 
                         self.numerical_aperture_radius_pixels, 
                         self.dx, 
                         tuple(self.N), 
                         tuple(self.M), 
                         self.device
                         )
        new_meta.natural_neighbor_weights = th.as_tensor(nnw)
        new_meta.beamlets = th.as_tensor(beamlets).to(new_meta.device)
        return new_meta
    
    

In [ ]:
#export 
@dataclass
class Metadata:
    k_max : np.array 
    alpha_rad : float
    E_ev : float 
    wavelength : float 
    rotation_deg : float 
    aberrations : np.array 
    sample_thickness_guess_angstrom : float
    
@dataclass
class Metadata4D(Metadata):
    scan_step : np.array 
    pixel_step : np.array 
    
    def __init__(self, E_ev=None, ):
        self.scan_step = np.array([0,0])
        self.k_max = np.array([0,0])
        self.alpha_rad = -1
        self.rotation_deg = 0
        self.E_ev = E_ev
        self.wavelength = -1
        self.aberrations = np.zeros((12,))
        self.pixel_step = np.array([0,0])
        self.sample_thickness_guess_angstrom = 0
        
    @staticmethod
    def from_dm4_file(filename):
        m = Metadata4D()
        with fileDM(filename) as f:
            m.E_ev = f.allTags['.ImageList.2.ImageTags.Microscope Info.Voltage']
            m.scan_step = np.array(f.scale[-2:]) * 10
        m.wavelength = wavelength(m.E_ev)
        return m 
    
    def load_voltage_and_scanstep_from_dm4_(self, filename):
        with fileDM(filename) as f:
            self.E_ev = f.allTags['.ImageList.2.ImageTags.Microscope Info.Voltage']
            self.scan_step = np.array(f.scale[-2:]) * 10
        self.wavelength = wavelength(self.E_ev)
        
    def to_h5(self, file_path, key = 'meta'):
        with h5py.File(file_path, 'a') as f:
            g = f.create_group(key)
            g.create_dataset('scan_step', data=self.scan_step)
            g.create_dataset('k_max', data=self.k_max)
            g.create_dataset('alpha_rad', data=self.alpha_rad)
            g.create_dataset('rotation_deg', data=self.rotation_deg)
            g.create_dataset('E_ev', data=self.E_ev)
            g.create_dataset('wavelength', data=self.wavelength)
            g.create_dataset('aberrations', data=self.aberrations)
            g.create_dataset('pixel_step', data=self.pixel_step)
            g.create_dataset('sample_thickness_guess_angstrom', data=self.sample_thickness_guess_angstrom)
            
    @staticmethod        
    def from_h5(file_path, key = 'meta'):
        res = Metadata4D()
        with h5py.File(file_path, 'r') as f:
            g = f[key]
            res.scan_step = g['scan_step'][...]
            res.pixel_step = g['pixel_step'][...]
            res.k_max = g['k_max'][...]
            res.alpha_rad = g['alpha_rad'][()]
            res.rotation_deg = g['rotation_deg'][()]
            res.E_ev = g['E_ev'][()]
            res.wavelength = g['wavelength'][()]
            res.aberrations = g['aberrations'][...]
            try:
                res.sample_thickness_guess_angstrom = g['sample_thickness_guess_angstrom'][()]
            except:
                pass
        return res 

In [ ]:
#export 
@dataclass
class Sparse3DData:
    __repr__ = basic_repr(['scan_dimensions', 'frame_dimensions'])
    def __init__(self, indices = None, counts = None, scan_dimensions = None, frame_dimensions = None):
        self.indices = indices
        self.counts = counts
        self.scan_dimensions = scan_dimensions 
        self.frame_dimensions = frame_dimensions 
        
    def __getitem__(self, slice):
        new_inds = self.indices[slice]
        new_cts = self.counts[slice]
        return Sparse3DData(new_inds, new_cts, (new_inds.shape[0],), self.frame_dimensions)
    
    def to(self, device, non_blocking = False):
        self.indices = self.indices.to(device, non_blocking=non_blocking)
        self.counts = self.counts.to(device, non_blocking=non_blocking)
        return self

In [ ]:
#export

def get_CoM(ar):
    """
    Finds and returns the center of mass of array ar.
    """
    nx, ny = np.shape(ar)
    ry, rx = np.meshgrid(np.arange(ny), np.arange(nx))
    tot_intens = np.sum(ar)
    xCoM = np.sum(rx * ar) / tot_intens
    yCoM = np.sum(ry * ar) / tot_intens
    return xCoM, yCoM

def get_probe_size(DP, thresh_lower=0.01, thresh_upper=0.99, N=100):
    """
    Gets the center and radius of the probe in the diffraction plane.

    The algorithm is as follows:
    First, create a series of N binary masks, by thresholding the diffraction pattern DP with a
    linspace of N thresholds from thresh_lower to thresh_upper, measured relative to the maximum
    intensity in DP.
    Using the area of each binary mask, calculate the radius r of a circular probe.
    Because the central disk is typically very intense relative to the rest of the DP, r should
    change very little over a wide range of intermediate values of the threshold. The range in which
    r is trustworthy is found by taking the derivative of r(thresh) and finding identifying where it
    is small.  The radius is taken to be the mean of these r values.
    Using the threshold corresponding to this r, a mask is created and the CoM of the DP times this
    mask it taken.  This is taken to be the origin x0,y0.

    Accepts:
        DP              (2D array) the diffraction pattern in which to find the central disk.
                        A position averaged, or shift-corrected and averaged, DP work well.
        thresh_lower    (float, 0 to 1) the lower limit of threshold values
        thresh_upper    (float, 0 to 1) the upper limit of threshold values
        N               (int) the number of thresholds / masks to use

    Returns:
        r               (float) the central disk radius, in pixels
        x0              (float) the x position of the central disk center
        y0              (float) the y position of the central disk center
    """
    thresh_vals = np.linspace(thresh_lower,thresh_upper,N)
    r_vals = np.zeros(N)

    # Get r for each mask
    DPmax = np.max(DP)
    for i in range(len(thresh_vals)):
        thresh = thresh_vals[i]
        mask = DP > DPmax*thresh
        r_vals[i] = np.sqrt(np.sum(mask)/np.pi)

    # Get derivative and determine trustworthy r-values
    dr_dtheta = np.gradient(r_vals)
    mask = (dr_dtheta <= 0) * (dr_dtheta >= 2*np.median(dr_dtheta))
    r = np.mean(r_vals[mask])

    # Get origin
    thresh = np.mean(thresh_vals[mask])
    mask = DP > DPmax*thresh
    x0,y0 = get_CoM(DP*mask)

    return r,x0,y0

class Sparse4DData:
    __repr__ = basic_repr(['scan_dimensions', 'frame_dimensions'])
    def __init__(self, indices = None, counts = None, scan_dimensions = None, frame_dimensions = None):
        self.indices = indices
        self.counts = counts
        self.scan_dimensions = scan_dimensions 
        self.frame_dimensions = frame_dimensions 
        
    def to_3d(self):
        ish = self.indices.shape
        _3d_view = (ish[0]*ish[1],ish[2])
        return Sparse3DData(th.as_tensor(self.indices).view(*_3d_view), 
                            th.as_tensor(self.counts).view(*_3d_view), 
                            (_3d_view[0],), 
                            self.frame_dimensions)
    @staticmethod
    def stack_vertical(data0, data1):
        no_count_indicator = np.iinfo(data0.indices.dtype).max
        inds_max = np.max([data0.indices.shape[-1], data1.indices.shape[-1]])
        inds_combined = np.zeros((data0.indices.shape[0] + data1.indices.shape[0], data0.indices.shape[1], inds_max),
                                 dtype=data0.indices.dtype)
        inds_combined[:] = no_count_indicator
        counts_combined = np.zeros((data0.counts.shape[0] + data1.counts.shape[0], data0.counts.shape[1], inds_max),
                                   dtype=data0.counts.dtype)
        inds_combined[:data0.indices.shape[0], :, :data0.indices.shape[2]] = data0.indices
        inds_combined[data0.indices.shape[0]:, :, :data1.indices.shape[2]] = data1.indices
        
        counts_combined[:data0.counts.shape[0], :, :data0.counts.shape[2]] = data0.counts
        counts_combined[data0.counts.shape[0]:, :, :data1.counts.shape[2]] = data1.counts
        
        inds_slice = inds_combined
        counts_slice = counts_combined
        
        data = Sparse4DData()
        data.counts = counts_slice
        data.indices = inds_slice
        data.frame_dimensions = data0.frame_dimensions.copy()
        data.scan_dimensions = np.array(data.counts.shape[:2])
        
        return data 
        
    @staticmethod
    def from_4Dcamera_file(filename, slice=None):
        with h5py.File(filename, 'r') as f0:
            frames = f0['electron_events/frames'][:]
            scan_dimensions = (f0['electron_events/scan_positions'].attrs['Ny'],
                               f0['electron_events/scan_positions'].attrs['Nx'])
            frame_dimensions = np.array((576, 576))
        
        def unragged_frames_size(frames):
            mm = 0
            for ev in frames:
                if ev.shape[0] > mm:
                    mm = ev.shape[0]
            return mm
        
        def make_unragged_frames(frames, scan_dimensions):
            unragged_frame_size = unragged_frames_size(frames.ravel())
            fr_full = np.zeros((frames.ravel().shape[0], unragged_frame_size), dtype=np.int32)
            fr_full[:] = np.iinfo(fr_full.dtype).max
            for ii, ev in enumerate(frames.ravel()):
                fr_full[ii, :ev.shape[0]] = ev
            fr_full_4d = fr_full.reshape((*scan_dimensions, fr_full.shape[1]))
            fr_full_4d = fr_full_4d[:, :-1, :] 
            return fr_full_4d
        
        d = Sparse4DData()
        d.indices = np.ascontiguousarray(make_unragged_frames(frames.ravel(), scan_dimensions))
        if slice is not None:            
            d.indices = d.indices[slice]
        d.scan_dimensions = np.array(d.indices.shape[:2])
        d.frame_dimensions = frame_dimensions
        d.counts = np.zeros(d.indices.shape, dtype=np.bool)
        d.counts[d.indices != np.iinfo(d.indices.dtype).max] = 1
        
        return d 
    
    def crop_symmetric_center_(self, center, max_radius = None):
        if max_radius is None:
            y_min_radius = np.min([center[0], self.frame_dimensions[0] - center[0]])
            x_min_radius = np.min([center[1], self.frame_dimensions[1] - center[1]])
            max_radius = np.min([y_min_radius, x_min_radius])
        inds = sp.to_device(self.indices,0)
        frame_dimensions = sp.to_device(self.frame_dimensions, 0)
        xp = sp.backend.get_array_module(inds)
        new_frames, new_frame_dimensions = crop_symmetric_around_center(inds,frame_dimensions, center, max_radius)
        print(f'old frames shape: {self.indices.shape}')
        print(f'new frames shape: {new_frames.shape}')
        print(f'old frames frame_dimensions: {self.frame_dimensions}')
        print(f'new frames frame_dimensions: {new_frame_dimensions}')
        self.indices = new_frames
        self.counts = np.zeros(self.indices.shape, dtype=np.bool)
        self.counts[self.indices != np.iinfo(self.indices.dtype).max] = 1
        self.frame_dimensions = new_frame_dimensions
        
    def crop_symmetric_center(self, center, max_radius = None, verbose = True):
        if max_radius is None:
            y_min_radius = np.min([center[0], self.frame_dimensions[0] - center[0]])
            x_min_radius = np.min([center[1], self.frame_dimensions[1] - center[1]])
            max_radius = np.min([y_min_radius, x_min_radius])
            
        inds = sp.to_device(self.indices,0)
        xp = sp.backend.get_array_module(inds)
        frame_dims = xp.array(self.frame_dimensions)
        new_frames, new_frame_dimensions = crop_symmetric_around_center(inds, frame_dims, center, max_radius)
        
        if verbose: 
            print(f'old frames shape: {self.indices.shape}')
            print(f'new frames shape: {new_frames.shape}')
            print(f'old frames frame_dimensions: {self.frame_dimensions}')
            print(f'new frames frame_dimensions: {new_frame_dimensions}')
            
        res = Sparse4DData()
        res.indices = new_frames
        res.counts = np.zeros(self.indices.shape, dtype=np.bool)
        res.counts[self.indices != np.iinfo(self.indices.dtype).max] = 1
        res.frame_dimensions = new_frame_dimensions
        res.scan_dimensions = self.scan_dimensions.copy()
        return res
    
    def rotate_(self, angle_rad, center=None):
        if center is None:
            center = self.frame_dimensions // 2
        new_indices = rotate(self.indices, self.frame_dimensions, center, angle_rad)
        self.indices = new_indices
    
    def rotate(self, angle_rad, center=None):
        if center is None:
            center = self.frame_dimensions // 2
        new_indices = rotate(self.indices, self.frame_dimensions, center, angle_rad)
        res = Sparse4DData()
        res.indices = new_indices
        res.counts = self.counts.copy()
        res.frame_dimensions = self.frame_dimensions
        res.scan_dimensions = self.scan_dimensions.copy()
        return res 
    
    def sum_diffraction(self):
        res = sum_frames(self.indices, self.counts, self.frame_dimensions)
        return res 
    
    @staticmethod
    def _determine_center_and_radius(data : Sparse4DData, manual=False, size=25, threshold=3e-1):
        sh = np.concatenate([data.scan_dimensions,data.frame_dimensions])
        c = np.zeros((2,))
        c[:] = (sh[-1] // 2, sh[-2] // 2)
        radius = np.ones((1,)) * sh[-1] // 2  
        inds = sp.to_device(data.indices[:size, :size].astype(np.uint32),0)
        cts = sp.to_device(data.counts[:size, :size].astype(np.uint32),0)
        xp = sp.backend.get_array_module(inds)
        dc_subset = sparse_to_dense_datacube_crop(inds,cts, (size,size), data.frame_dimensions, c, radius, bin=2)
        dcs = xp.sum(dc_subset, (0, 1))
        m1 = dcs.get()
        m = (gaussian(m1.astype(np.float32),2) > m1.max() * threshold).astype(np.float)
        r, y0, x0 = get_probe_size(m)
        return 2 * np.array([y0,x0]), r*2
    
    def determine_center_and_radius(self, manual=False, size=25, threshold=3e-1):
        return Sparse4DData._determine_center_and_radius(self, manual, size=size, threshold=threshold)
    
    def to_dense(self, bin_factor, n_batches=4):
        dense = sparse_to_dense_datacube_crop_gain_mask(self.indices, self.counts.astype(np.int16), self.scan_dimensions,
                                                self.frame_dimensions, self.frame_dimensions/2, self.frame_dimensions[0]/2,
                                                self.frame_dimensions[0]/2, binning=bin_factor, n_batches=n_batches, fftshift=False)
        return dense
    
    @staticmethod
    def from_dense(dense, make_float = False, n_batches=1):
        res = Sparse4DData()
        res.frame_dimensions = np.array(dense.shape[-2:])
        res.scan_dimensions = np.array(dense.shape[:2])
        
        cuda.select_device(1)
        dev = th.device('cuda:1')

        inds = np.prod(res.frame_dimensions)
        if inds > 2**31:
            dtype = th.int64
        elif inds > 2**15:
            dtype = th.int32
        elif inds > 2**8:
            dtype = th.int16
        else:
            dtype = th.uint8
        
        nonzeros = np.sum((dense > 0),(2,3))
        nonzeros = np.max(nonzeros)

        bits_counts = np.log2(dense.max())
        if make_float:
            dtype_counts = th.float32
        else:
            if bits_counts > np.log2(2**31-1):
                dtype_counts = th.int64
            elif bits_counts > np.log2(2**15-1):
                dtype_counts = th.int32
            elif bits_counts > 8:
                dtype_counts = th.int16
            else:
                dtype_counts = th.uint8
                
        print(f'Using dtype: {dtype} for indices')
        print(f'Using dtype: {dtype_counts} for counts')

        threadsperblock = (16, 16)
        blockspergrid = tuple(np.ceil(res.scan_dimensions / threadsperblock).astype(np.int32))
        
        ds = dense.shape
        K = dense.shape[0]
        divpts = array_split_divpoints_ntotal(K, n_batches)
    
        dense = th.as_tensor(dense) 
        indices = th.zeros((*dense.shape[:2], nonzeros), dtype=dtype)
        counts = th.zeros((*dense.shape[:2], nonzeros), dtype=dtype_counts)
    
        indices[:] = th.iinfo(dtype).max
        fd = th.tensor(res.frame_dimensions).to(dev)
        for b in range(n_batches):
        
            denseb = dense[divpts[b]:divpts[b + 1]].to(dev)
            indicesb = indices[divpts[b]:divpts[b + 1]].to(dev)
            indicesb[:] = th.iinfo(dtype).max
            countsb = counts[divpts[b]:divpts[b + 1]].to(dev)
            
            dense_to_sparse_kernel[blockspergrid, threadsperblock](denseb, indicesb, countsb, fd)
            
            counts[divpts[b]:divpts[b + 1]] = countsb.cpu()
            indices[divpts[b]:divpts[b + 1]] = indicesb.cpu()

        res.indices = indices.numpy()
        res.counts = counts.numpy()
        
        print(f'frame_dimensions: {res.frame_dimensions}')
        print(f'scan_dimensions : {res.scan_dimensions}')
        
        cuda.select_device(0)
        
        return res
    
    def fill_missing_data_(self):
        data = self
        import sigpy as sp
        import cupy as cp
        from numba import cuda
    
        empty_diffraction = np.sum(data.counts, 2) == 0
        if empty_diffraction.sum() > 0:
            empty_diffraction_indices = np.array(np.nonzero(empty_diffraction)).T
            neighbors = np.stack([
                empty_diffraction_indices + [1, 0],
                empty_diffraction_indices + [1, 1],
                empty_diffraction_indices + [0, 1],
                empty_diffraction_indices + [-1, 0],
                empty_diffraction_indices + [-1, -1],
                empty_diffraction_indices + [0, -1],
                empty_diffraction_indices + [1, -1],
                empty_diffraction_indices + [-1, 1]
            ])
            eds = neighbors.shape
            nb = neighbors.reshape((eds[0] * eds[1], 2))
            nb[nb[:, 0] >= data.frame_dimensions[0], 0] = data.frame_dimensions[0] - 1
            nb[nb[:, 1] >= data.frame_dimensions[1], 1] = data.frame_dimensions[1] - 1
            nb[nb[:, 0] < 0, 0] = 0
            nb[nb[:, 1] < 0, 1] = 0
            cts_neighbors = data.counts[nb[:, 0], nb[:, 1]]
            idx_neighbors = data.indices[nb[:, 0], nb[:, 1]]
        
            @cuda.jit
            def sparse_to_dense_datacube_kernel_crop(dc, indices, counts, frame_dimensions, no_count_indicator):
                n = cuda.grid(1)
                N, MYBIN, MXBIN = dc.shape
                MY, MX = frame_dimensions
                if n < N:
                    for i in range(indices[n].shape[0]):
                        idx1d = indices[n, i]
                        my = int(idx1d // MX)
                        mx = int(idx1d - my * MX)
                        n = int(1.0 * n)
                        if idx1d != no_count_indicator:
                            cuda.atomic.add(dc, (n, my, mx), counts[n, i])
        
        
            def sparse_to_dense_datacube_crop(indices, counts, frame_dimensions):
                xp = sp.backend.get_array_module(indices)
                dc = xp.zeros((indices.shape[0], *frame_dimensions), dtype=counts.dtype)
                print(dc.shape)
                threadsperblock = (128,)
                blockspergrid = tuple(np.ceil(np.array(indices.shape[0]) / threadsperblock).astype(np.int32))
                no_count_indicator = np.iinfo(indices.dtype).max
                sparse_to_dense_datacube_kernel_crop[blockspergrid, threadsperblock](dc, indices, counts,
                                                                                     xp.array(frame_dimensions),
                                                                                     no_count_indicator)
                return dc
        
        
            @cuda.jit
            def dense_to_sparse_kernel(dense, indices, counts, frame_dimensions):
                n = cuda.grid(1)
                N, MYBIN, MXBIN = dense.shape
                MY, MX = frame_dimensions
                if n < N:
                    k = 0
                    for mx in range(MX):
                        for my in range(MY):
                            idx1d = my * MX + mx
                            if dense[n, my, mx] > 0:
                                indices[n, k] = idx1d
                                counts[n, k] = dense[n, my, mx]
                                k += 1
        
            dense = sparse_to_dense_datacube_crop(cp.array(idx_neighbors), cp.array(cts_neighbors).astype(np.float32),
                                                  data.frame_dimensions)
            dense = dense.reshape((eds[0], eds[1], *data.frame_dimensions))
            replace = cp.round_(cp.mean(dense, 0)).astype(cp.uint8)
        
            indices_replace = cp.zeros((eds[1], data.counts.shape[2]), dtype=data.indices.dtype)
            indices_replace[:] = np.iinfo(data.indices.dtype).max
            counts_replace = cp.zeros((eds[1], data.counts.shape[2]), dtype=data.counts.dtype)
            threadsperblock = (128,)
            blockspergrid = tuple(np.ceil(np.array(eds[1]) / threadsperblock).astype(np.int32))
            dense_to_sparse_kernel[blockspergrid, threadsperblock](replace, indices_replace, counts_replace,
                                                                   cp.array(data.frame_dimensions))
            replace_inds = empty_diffraction_indices.T
            data.counts[replace_inds[0], replace_inds[1], :] = counts_replace.get()
            data.indices[replace_inds[0], replace_inds[1], :] = indices_replace.get()
    
    @staticmethod
    def rebin(sparse_data: Sparse4DData, bin_factor : int, n_batches=4) -> Sparse4DData:
        dense = sparse_to_dense_datacube_crop_gain_mask(sparse_data.indices, sparse_data.counts.astype(np.int16), sparse_data.scan_dimensions, 
                                                sparse_data.frame_dimensions, sparse_data.frame_dimensions/2, sparse_data.frame_dimensions[0]/2,
                                                sparse_data.frame_dimensions[0]/2, binning=bin_factor, n_batches=n_batches, fftshift=False)
        sparse = Sparse4DData.from_dense(dense)
        return sparse
    
    @staticmethod
    def fftshift(sparse_data: Sparse4DData) -> Sparse4DData:
        indices = sparse_data.indices
        scan_dimensions = sparse_data.scan_dimensions
        frame_dimensions = sparse_data.frame_dimensions
        center_frame = frame_dimensions / 2
        
        threadsperblock = (16, 16)
        blockspergrid = tuple(np.ceil(np.array(indices.shape[:2]) / threadsperblock).astype(np.int32))
    
        no_count_indicator = np.iinfo(indices.dtype).max
        
        inds = sp.to_device(indices, 0)
        center_frame = sp.to_device(center_frame, 0)
        scan_dimensions = sp.to_device(scan_dimensions, 0)
        
        fftshift_kernel[blockspergrid, threadsperblock](inds, center_frame, scan_dimensions, no_count_indicator)
        
        res = Sparse4DData()
        res.indices = inds.get()
        res.counts = sparse_data.counts.copy()
        res.scan_dimensions = sparse_data.scan_dimensions.copy()
        res.frame_dimensions = sparse_data.frame_dimensions.copy()
        
        return res
    
    @staticmethod
    def fftshift_and_pad_to(sparse_data: Sparse4DData, pad_to_frame_dimensions) -> Sparse4DData:
        indices = sparse_data.indices
        scan_dimensions = sparse_data.scan_dimensions
        frame_dimensions = sparse_data.frame_dimensions
        center_frame = frame_dimensions / 2

        threadsperblock = (16, 16)
        blockspergrid = tuple(np.ceil(np.array(indices.shape[:2]) / threadsperblock).astype(np.int32))

        no_count_indicator_old = np.iinfo(indices.dtype).max
        center_frame = sp.to_device(center_frame,0)
        xp = sp.backend.get_array_module(center_frame)

        inds = np.prod(pad_to_frame_dimensions)
        if inds > 2**15:
            dtype = xp.int64
        elif inds > 2**15:
            dtype = xp.int32
        elif inds > 2**8:
            dtype = xp.int16
        else:
            dtype = xp.uint8

        no_count_indicator_new = xp.iinfo(dtype).max

        inds = sp.to_device(indices,0).astype(dtype)
        pad_to_frame_dimensions = sp.to_device(pad_to_frame_dimensions,0)
        
        scan_dimensions = sp.to_device(scan_dimensions,0)
        
        fftshift_pad_kernel[blockspergrid, threadsperblock](inds, center_frame, scan_dimensions,  pad_to_frame_dimensions, 
                                                            no_count_indicator_old, no_count_indicator_new)
        sparse_data.indices = inds.get()
        sparse_data.frame_dimensions = pad_to_frame_dimensions.get()
        return sparse_data
    
    def fftshift_(self):
        indices = self.indices
        scan_dimensions = self.scan_dimensions
        frame_dimensions = self.frame_dimensions
        center_frame = frame_dimensions / 2
        
        threadsperblock = (16, 16)
        blockspergrid = tuple(np.ceil(np.array(indices.shape[:2]) / threadsperblock).astype(np.int32))
    
        no_count_indicator = np.iinfo(indices.dtype).max
        
        inds = sp.to_device(indices, 0)
        center_frame = sp.to_device(center_frame, 0)
        scan_dimensions = sp.to_device(scan_dimensions, 0)
        
        fftshift_kernel[blockspergrid, threadsperblock](inds, center_frame, scan_dimensions, no_count_indicator)
        self.indices = inds.get()
        return self
    
    def fftshift_and_pad_to_(self, pad_to_frame_dimensions):
        return Sparse4DData.fftshift_and_pad_to(self, pad_to_frame_dimensions)
    
    def bin(self, binning_factor, n_batches=4):
        res = Sparse4DData.rebin(self, binning_factor, n_batches=n_batches)                
        return res 
    
    def virtual_annular_image(self, inner_radius, outer_radius, center):
        inds = sp.to_device(self.indices, 0)
        xp = sp.backend.get_array_module(inds)
        cts = xp.array(self.counts, dtype=xp.uint32)
        ctr = xp.array(center)
        frame_dims = xp.array(self.frame_dimensions)
        img = xp.zeros(tuple(self.scan_dimensions), dtype=xp.uint32)
        no_count_indicator = xp.iinfo(self.indices.dtype).max
        threadsperblock = (16, 16)
        blockspergrid = tuple(np.ceil(np.array(self.indices.shape[:2]) / threadsperblock).astype(np.int32))
        virtual_annular_image_kernel[blockspergrid, threadsperblock](img, inds, cts, inner_radius, outer_radius, ctr, 
                                                                     frame_dims, no_count_indicator)
        return img.get()
    
    def fluence(self, dr):
        sum_electrons = self.counts.sum()
        area = np.prod(self.scan_dimensions) * dr**2
        return sum_electrons/area

    def flux(self, dr, dwell_time):
        fluence = self.fluence(dr)
        flux = fluence / (np.prod(self.scan_dimensions) * dwell_time)
        return flux 
    
    def slice(self, slice):
        res = Sparse4DData()
        res.indices = np.ascontiguousarray(self.indices[slice]) 
        res.counts = np.ascontiguousarray(self.counts[slice])
        res.scan_dimensions = np.array(res.counts.shape[:2])
        res.frame_dimensions = self.frame_dimensions.copy()     
        return res 
    
    def center_of_mass(self: Sparse4DData):
        qx, qy = np.meshgrid(np.arange(self.scan_dimensions[0]),np.arange(self.scan_dimensions[1]))    
        comx = th.zeros(tuple(self.scan_dimensions), dtype=th.float32)
        comy = th.zeros(tuple(self.scan_dimensions), dtype=th.float32)
        
        no_count_indicator = np.iinfo(self.indices.dtype).max
        
        mass = np.sum(self.counts,2)
        
        threadsperblock = (16, 16)
        blockspergrid = tuple(np.ceil(np.array(self.indices.shape[:2]) / threadsperblock).astype(np.int32))
        
        qx = th.tensor(qx).to(th.float32)
        qy = th.tensor(qy).to(th.float32)
        center_of_mass_kernel[blockspergrid, threadsperblock](comx, comy, th.tensor(self.indices), th.tensor(self.counts.astype(np.float32)), 
                                                              th.tensor(self.frame_dimensions), no_count_indicator, qx, qy)
        comy = comy.get()
        comx = comx.get()
        comx /= mass + 1e-6
        comy /= mass + 1e-6
        comy[comy==0] = np.mean(comy[comy!=0])
        comx[comx==0] = np.mean(comx[comx!=0])
        comx -= np.mean(comx)
        comy -= np.mean(comy)
        return comy, comx    
    
    def to_h5(self, file_path, key):
        with h5py.File(file_path, 'a') as f:
            g = f.create_group(key)
            g.create_dataset('counts', data=self.counts, compression="gzip", compression_opts=7)
            g.create_dataset('indices', data=self.indices, compression="gzip", compression_opts=7)
            g.create_dataset('frame_dimensions', data=self.frame_dimensions)
            g.create_dataset('scan_dimensions', data=self.scan_dimensions)
    
    @staticmethod        
    def from_h5(file_path, key):
        res = Sparse4DData()
        with h5py.File(file_path, 'r') as f:
            g = f[key]
            res.counts = g['counts'][...]
            res.indices = g['indices'][...]
            res.frame_dimensions = g['frame_dimensions'][...]
            res.scan_dimensions = g['scan_dimensions'][...]
        return res

In [ ]:
#export 
class Dense4DDataset(Dataset):
    data : th.tensor
    scan_dimensions : np.array
    scan_positions : int
    frame_dimensions : np.array
    
    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, index) -> th.tensor:
        return self.data[index]

    def __init__(self, data: Union[np.array, th.tensor]):
        dsh = data.shape
        self.data = th.as_tensor(data).view(dsh[0]*dsh[1],dsh[2],dsh[3])
        self.scan_positions = self.data.shape[0]
        self.scan_dimensions = np.array(dsh[:2])
        self.frame_dimensions = np.array(dsh[2:])

In [ ]:
#export   

class LinearIndexEncoded4DDataset(Dataset):
    sparse_data : Sparse3DData
    scan_dimensions : np.array
    scan_positions : int
    frame_dimensions : np.array
    
    def __len__(self):
        return self.sparse_data.scan_dimensions[0]

    def __getitem__(self, index) -> Sparse3DData:
        return self.sparse_data[index]

    def __init__(self, sparse_data: Sparse4DData):
        self.sparse_data = sparse_data.to_3d()
        self.scan_positions = self.sparse_data.scan_dimensions
        self.scan_dimensions = sparse_data.scan_dimensions
        self.frame_dimensions = sparse_data.frame_dimensions
        

